In [4]:
import gradio as gr
import dill
import pandas as pd
import xgboost as xgb
import numpy as np
import pickle

c:\Users\yaman\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def decode_file(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

model = decode_file('model.pkl')

In [6]:
def predict(gender, age, hypertension, ever_married, work_type, heart_disease, avg_glucose_level, bmi, smoking_status, Residence_type):
    # Mapping for categorical variables
    gender_mapping = {'Male': 1, 'Female': 0}
    hypertension_mapping = {'Yes': 1, 'No': 0}
    ever_married_mapping = {'Yes': 1, 'No': 0}
    work_type_mapping = {'Private': 2, 'Self-employed': 4, 'Govt_job': 3, 'Children': 1, 'Never_worked': 0}
    heart_disease_mapping = {'Yes': 1, 'No': 0}
    smoking_status_mapping = {'formerly smoked': 3, 'Smokes': 1, 'Never smoked': 2, 'Unknown': 0}
    Residence_type_mapping = {'Urban': 1, 'Rural': 0}

    # Map categorical variables to their corresponding numerical values
    gender = gender_mapping[gender]
    hypertension = hypertension_mapping[hypertension]
    ever_married = ever_married_mapping[ever_married]
    work_type = work_type_mapping[work_type]
    heart_disease = heart_disease_mapping[heart_disease]
    smoking_status = smoking_status_mapping[smoking_status]
    Residence_type = Residence_type_mapping[Residence_type]

    inputs = [gender, age, hypertension, ever_married, work_type, heart_disease, avg_glucose_level, bmi, smoking_status, Residence_type]
    input_labels = ['gender', 'age', 'hypertension', 'ever_married', 'work_type', 'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status', 'Residence_type']

    # Convert the input into a pandas DataFrame
    input_df = pd.DataFrame([inputs], columns=input_labels)

    # Predict the stroke probability
    prediction = model.predict_proba(input_df)[0][1]

    # Return the prediction
    result = "The probability of stroke is {:.2f}%".format(prediction * 100)  # to give a percentage
    return result


In [7]:
predict('Female',57.0,'No',"Yes",'Private',"No",82.54,33.4,'Unknown','Rural')


'The probability of stroke is 3.97%'

In [8]:

input_labels = [
    'gender', 'age', 'hypertension', 'ever_married', 'work_type', 
    'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status', 'Residence_type'
    ]

inputs = [
    gr.components.Dropdown(['Male', 'Female'], label="gender"),
    gr.components.Number(label="age"),
    gr.components.Dropdown(['Yes', 'No'], label="Hypertension"),
    gr.components.Dropdown(['Yes', 'No'], label="Ever Married"),
    gr.components.Dropdown(['Private', 'Self-employed', 'Govt_job', 'Children', 'Never_worked'], label="Work Type"),
    gr.components.Dropdown(['Yes', 'No'], label="Heart Disease"),
    gr.components.Number(label="average glucose level"),
    gr.components.Number(label="bmi"),
    gr.components.Dropdown(['formerly smoked','Smokes', 'Never smoked', 'Unknown'], label="Smoking Status"),
    gr.components.Dropdown(['Urban', 'Rural'], label="Residence Type"),
]

iface = gr.Interface(
    fn=predict,
    inputs=inputs,
    outputs="text",
    title="Prediction Model",
    description="Enter the required information to make a prediction.",
    theme="default"
)

iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
